# 大二基础属性分析

针对微博账号 拯救大龄二次元 里面的投稿进行的一个基础属性的简单文本分析。

数据时间 2018.1.1到2022.2.9中午

仅包括原创微博投稿（即后续转发不算做在内，第二次投稿如果为新的原创微博则包括）的文本信息，不包括图像信息，已经删除的不算做在内。

爬虫由https://github.com/dataabc/weiboSpider实现

分词采用结巴分词https://github.com/fxsjy/jieba

词云采用wordcloud

包括位置 ID 性别 出生年份，身高体重，学历，工作，发布时间，点赞数，评论数，转发数

不是专业做这个的，所以文本分析部分可能会有一部分错误，欢迎一起来改进这个项目。

## 加载 预处理

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings ('ignore')
os.chdir('E:\BigOtaku\拯救大龄二次元')
Full_text=pd.read_csv('1855501681.csv')
print(Full_text.shape)
Full_text.head()

(2866, 11)


,微博id,微博正文,头条文章url,原始图片url,微博视频url,发布位置,发布时间,发布工具,点赞数,转发数,评论数
0,KCPLgwOlS,[置顶]我又双叒叕来宠粉了！！快来成为我的真爱粉，享受真爱粉专属标识，评论靠前，还能加入真爱...,NaN,无,无,无,2021-10-15 13:26,iPhone 8 Plus,130,0,33
1,LeF5YCq4I,#魔都求脱单# 投稿ID：@十六夜久苑 男，89年，176/71，土著，日企，详见长微博 ...,NaN,http://ww4.sinaimg.cn/large/6e98b571gy1gz5smdt...,无,无,2022-02-09 12:54,新版微博 weibo.com,2,0,1
2,LeEVykKCh,#湖南邵阳求脱单# 投稿ID：@Joekeny 男，99年，172/112，本科，体制内，...,NaN,http://ww1.sinaimg.cn/large/6e98b571gy1gz5sv6j...,无,无,2022-02-09 12:28,微博 weibo.com,96,0,36
3,LeE8Qrc6A,#魔都求脱单# 投稿ID：@一颗疯芒精 女，99年，170/52，本科，外企部门助理，详见...,NaN,http://ww1.sinaimg.cn/large/6e98b571gy1gz5sp35...,无,无,2022-02-09 10:29,微博 weibo.com,154,0,77
4,LeDm8xDq2,#杭州求脱单# 投稿ID：@一杯猫猫w 女，96年，身高168/体重55-58之间徘徊（，...,NaN,http://ww4.sinaimg.cn/large/6e98b571gy1gz3xk9p...,无,无,2022-02-09 08:29,微博 weibo.com,275,5,122


一共2866条微博，这里面包括一些号主本人的，然后还要清理掉一些不需要的属性，例如头条文章url， 原始图片url	微博视频url	发布位置，以及爬虫过程中可能重复爬下来的

In [5]:
Full_text.drop_duplicates(inplace=True)
print(Full_text.duplicated().sum())

0


In [6]:
text_new=Full_text.drop('头条文章url',axis=1)
text_new=text_new.drop('原始图片url',axis=1)
text_new=text_new.drop('发布位置',axis=1)
text_new=text_new.drop('发布工具',axis=1)
text_new=text_new.drop('微博id',axis=1)
text_new=text_new.drop('微博视频url',axis=1)

In [7]:
text_new

,微博正文,发布时间,点赞数,转发数,评论数
0,[置顶]我又双叒叕来宠粉了！！快来成为我的真爱粉，享受真爱粉专属标识，评论靠前，还能加入真爱...,2021-10-15 13:26,130,0,33
1,#魔都求脱单# 投稿ID：@十六夜久苑 男，89年，176/71，土著，日企，详见长微博 ...,2022-02-09 12:54,2,0,1
2,#湖南邵阳求脱单# 投稿ID：@Joekeny 男，99年，172/112，本科，体制内，...,2022-02-09 12:28,96,0,36
3,#魔都求脱单# 投稿ID：@一颗疯芒精 女，99年，170/52，本科，外企部门助理，详见...,2022-02-09 10:29,154,0,77
4,#杭州求脱单# 投稿ID：@一杯猫猫w 女，96年，身高168/体重55-58之间徘徊（，...,2022-02-09 08:29,275,5,122
...,...,...,...,...,...
2861,好多人喜欢替基友操心，来问我能不能代投。统一回答一下平台原则上是不接受代投的。一来，求脱团这...,2018-01-23 16:39,41,2,75
2862,现在玩家的操作越来越骚了，第一次遇到男女朋友吵架闹分，然后女生写了个相亲稿来投给我，之后把审...,2018-01-15 12:37,354,141,297
2863,我这里收到的绿帽类的投稿不算多，但人间蒸发玩消失的真不少。喜欢二次元，又沉迷游戏，突然某一天...,2018-01-06 21:24,251,88,440
2864,#杭州求脱团# 投稿ID：@凝心_电气老鼠LOVE 女，89年，161cm。日本硕士毕业回...,2018-01-05 19:00,40,49,49


In [8]:
new_text_list = text_new['微博正文'].apply(lambda x: x.startswith('#'))

In [9]:
text_clean = text_new[new_text_list ]

In [10]:
text_clean

,微博正文,发布时间,点赞数,转发数,评论数
1,#魔都求脱单# 投稿ID：@十六夜久苑 男，89年，176/71，土著，日企，详见长微博 ...,2022-02-09 12:54,2,0,1
2,#湖南邵阳求脱单# 投稿ID：@Joekeny 男，99年，172/112，本科，体制内，...,2022-02-09 12:28,96,0,36
3,#魔都求脱单# 投稿ID：@一颗疯芒精 女，99年，170/52，本科，外企部门助理，详见...,2022-02-09 10:29,154,0,77
4,#杭州求脱单# 投稿ID：@一杯猫猫w 女，96年，身高168/体重55-58之间徘徊（，...,2022-02-09 08:29,275,5,122
5,#今天你投稿了么# 粉丝头条每加1万阅读需要支付200，10万就是2000，我这发个帖子人均...,2022-02-09 08:00,453,5,140
...,...,...,...,...,...
2856,#粉丝投稿# #粉丝吐槽#辛苦大二当个树洞了。因为是母胎solo，所以以为自己并不颜控，后来...,2018-02-11 15:42,83,31,235
2857,#粉丝投稿# #粉丝吐槽# 帖子求删另想吐槽一个事，希望大家在勾搭之前先和家人沟通好择偶标准...,2018-02-06 13:52,68,10,219
2858,#北海求脱团# 投稿ID：@湫洋sama 因为老家在北海所以北海带进了tag里，工作坐标柳...,2018-02-05 12:30,24,1,73
2859,#今天你投稿了么# “不想投的人不管怎么催都不会投”，这点道理我还是清楚的。还年轻、有资本、...,2018-01-31 08:00,53,5,502


简单筛选之后发现还剩一些 需要使用正则表达式

In [11]:
new_text_list2=text_clean['微博正文'].str.contains('投稿ID')

In [12]:
text_clean=text_clean[new_text_list2]
text_clean

,微博正文,发布时间,点赞数,转发数,评论数
1,#魔都求脱单# 投稿ID：@十六夜久苑 男，89年，176/71，土著，日企，详见长微博 ...,2022-02-09 12:54,2,0,1
2,#湖南邵阳求脱单# 投稿ID：@Joekeny 男，99年，172/112，本科，体制内，...,2022-02-09 12:28,96,0,36
3,#魔都求脱单# 投稿ID：@一颗疯芒精 女，99年，170/52，本科，外企部门助理，详见...,2022-02-09 10:29,154,0,77
4,#杭州求脱单# 投稿ID：@一杯猫猫w 女，96年，身高168/体重55-58之间徘徊（，...,2022-02-09 08:29,275,5,122
6,#北京/上海求脱单# 投稿ID：@夏梨- 女，94年，165/49，北京土著，近两年工作在...,2022-02-08 22:00,420,13,310
...,...,...,...,...,...
2844,#济南求脱团# 投稿ID：@喜欢吃零食的小牧君 男，89年，身高176微胖，房地产开发公...,2018-03-14 12:38,9,1,51
2845,#北京求脱团# 投稿ID：@Lid丹丹 八六年生人，女，帝都宣武土著，狮子座，血型B型，身...,2018-03-11 13:15,135,83,202
2846,#悉尼求脱团# 可异地，投稿ID：@钢琴上的螃蟹 男，86年，180/65，天津人，现居...,2018-03-04 12:15,72,18,277
2858,#北海求脱团# 投稿ID：@湫洋sama 因为老家在北海所以北海带进了tag里，工作坐标柳...,2018-02-05 12:30,24,1,73


一共2864条数据

## 地点分析

In [60]:
loc=text_clean['微博正文'].str.extract('#(.+)投稿ID',expand=True)
loc

,0
1,魔都求脱单#
2,湖南邵阳求脱单#
3,魔都求脱单#
4,杭州求脱单#
6,北京/上海求脱单#
...,...
2844,济南求脱团#
2845,北京求脱团#
2846,悉尼求脱团# 可异地，
2858,北海求脱团#


In [51]:
text = " ".join(i for i in data.text)
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)
plt.figure( figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

NameError: name 'data' is not defined